<a href="https://colab.research.google.com/github/kjahan/recursive_lm/blob/main/notebooks/ai_detector_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Can we use GPT to generate text that doesn't get detected as AI text?

There are a number of models and tools out there that help people to detect if a text is written by AI or not! See below:

## AI Text Detection Models

Can we build a system that make ai text to not be detectable?

https://www.kaggle.com/code/guslovesmath/hugging-face-ai-text-detection

https://huggingface.co/tommyliphys/ai-detector-distilbert

#### SuperAnnotate

https://huggingface.co/SuperAnnotate/ai-detector

GPT Zero:

https://app.gptzero.me/app/ai-scan?aiDocumentId=2bbcfc30-4208-4617-ace2-6386091a1030&nexus=true&isAnonymous=true

## Humanize

Aman showed me this site and he said he uses it for his assignments:

https://www.humanizeai.pro/

https://aithor.com/

https://www.reddit.com/r/ChatGPT/comments/1da5ben/can_i_train_chat_gpt_to_humanize_ai_text_like/

https://www.rephrasy.ai/

https://community.openai.com/t/how-do-i-humanize-ai-text-and-get-100-human-score/700565/3

https://gpthuman.ai/

### Victor comment

Victor asked me if you can fundementally use GPT to change GPT language distribution. The answer is no!

Can we use a praphraser?

https://huggingface.co/eugenesiow/bart-paraphrase

In [ ]:
!pip install git+https://github.com/superannotateai/generated_text_detector.git@v1.0.0

  Cloning https://github.com/superannotateai/generated_text_detector.git (to revision v1.0.0) to /tmp/pip-req-build-lfv697lr
  Running command git clone --filter=blob:none --quiet https://github.com/superannotateai/generated_text_detector.git /tmp/pip-req-build-lfv697lr
  Running command git checkout -q ad27066a2c26d8fc9477ebfd8458a51b4b78e664
  Resolved https://github.com/superannotateai/generated_text_detector.git to commit ad27066a2c26d8fc9477ebfd8458a51b4b78e664
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [ ]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [ ]:
from generated_text_detector.utils.model.roberta_classifier import RobertaClassifier
from transformers import AutoTokenizer
import torch.nn.functional as F

import openai
from openai import OpenAI

import getpass
import re
import os
import glob
import time

import numpy as np
from numpy import linalg as LA

from io import StringIO
from contextlib import redirect_stdout

## Load model

In [ ]:
model = RobertaClassifier.from_pretrained("SuperAnnotate/ai-detector")
tokenizer = AutoTokenizer.from_pretrained("SuperAnnotate/ai-detector")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/274 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

## Test AI text detector as evaluator

Prompting GPT-4o in chatpt to write "Write one paragraph about BERT."


In [16]:
text = "BERT (Bidirectional Encoder Representations from Transformers) is a deep learning model developed by Google for natural language processing (NLP). Unlike traditional language models that process text sequentially, BERT uses a transformer architecture to consider context from both directions (left and right) of each word in a sentence. This bidirectional approach allows BERT to better understand the meaning of words based on surrounding words, improving its performance on a wide range of NLP tasks, such as question answering, sentiment analysis, and named entity recognition. BERT is pre-trained on vast amounts of text and can be fine-tuned for specific tasks, making it highly versatile and a state-of-the-art model for many NLP applications."

tokens = tokenizer.encode_plus(
text,
add_special_tokens=True,
max_length=512,
padding='longest',
truncation=True,
return_token_type_ids=True,
return_tensors="pt"
)

_, logits = model(**tokens)

proba = F.sigmoid(logits).squeeze(1).item()

print(proba)

0.750756561756134


## Human text copied from Reddit

https://www.reddit.com/r/MLQuestions/comments/1aq2y9q/can_someone_explain_bert_to_me_i_am_having/

In [17]:
text = """I was in the same boat not long ago, and struggled to understand what Bert really does. There is an awesome intuitive article by Jay alamar (I forgot the spelling, you can Google it) which helped a ton.
But in essence, bert learns what words are supposed to go together based on their statistical frequency of showing up together in specific orders in the texts that the model is trained on. It is good for things like classification and recommendation.
A very simplified example is that if the training texts have the patterns " he ate the fish", "he ate the fish", " he ate the cheese", Bert will learn these patterns and then if you ask the model what words cheese is similar to, it may respond with fish because it appears in the same contexts frequently."""

tokens = tokenizer.encode_plus(
text,
add_special_tokens=True,
max_length=512,
padding='longest',
truncation=True,
return_token_type_ids=True,
return_tensors="pt"
)

_, logits = model(**tokens)

proba = F.sigmoid(logits).squeeze(1).item()

print(proba)

0.015270763076841831


## Open AI API Keys

In [ ]:
try:
    openai.api_key = getpass.getpass()
except Exception as error:
    print('ERROR', error)

··········


## Models

https://platform.openai.com/docs/models

In [ ]:
#MODEL = "gpt-3.5-turbo"
# MODEL = "gpt-4o" #"gpt-4" #"gpt-4-0613"
MODEL = "o1"

## Recursion with feedback doesnt seem working

I used higher temperature !

In [ ]:
system_prompt = "You are a helpful assistant. I'll give you a topic. Your job is to write a parapgraph about it which looks like a human writing."
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai.api_key,
)

base_user_prompt = "Write one paragraph about BERT."

cnt = 0
generated_texts = []
probs = []

while True:
  start_time = time.time()

  if cnt == 0:
    user_prompt = base_user_prompt

  #print(user_prompt)

  response = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt}
      ],
      temperature=0.5,
  )
  text = response.choices[0].message.content.strip()
  print(f"Runtime: {round((time.time() - start_time), 2)}")
  generated_texts.append(text)


  tokens = tokenizer.encode_plus(
  text,
  add_special_tokens=True,
  max_length=512,
  padding='longest',
  truncation=True,
  return_token_type_ids=True,
  return_tensors="pt"
  )

  _, logits = model(**tokens)

  prob = F.sigmoid(logits).squeeze(1).item()

  print(f"\n{text}")
  print(f"AI likelihood: {prob}")
  probs.append(prob)
  cnt += 1
  if cnt >= 1:
    user_prompt = "These are some texts that you have generated along with their likelihood for the text to be detected as AI text. Given all below examples, can you write a new paragraph about BERT which is more like human than the following ones?"
    for inx in range(len(generated_texts)):
      user_prompt += "\ntext: {}".format(generated_texts[inx])
      user_prompt += "\nai likelihood: {}%\n\n".format(round(100*probs[inx], 3))

  if cnt > 5 or probs[-1] < 0.1:
    break


Runtime: 2.1

BERT, which stands for Bidirectional Encoder Representations from Transformers, is a groundbreaking model developed by Google that has significantly advanced the field of natural language processing (NLP). Unlike previous models that processed text in a unidirectional manner, BERT is designed to understand the context of a word based on the words that come before and after it, thanks to its bidirectional approach. This allows BERT to capture the nuances of language more effectively, leading to improvements in tasks such as question answering and language inference. Since its release, BERT has become a foundational model for many NLP applications and has inspired numerous variations and improvements within the AI community. Its ability to pre-train on large datasets and fine-tune on specific tasks has made it an invaluable tool for both researchers and practitioners seeking to enhance machine understanding of human language.
AI likelihood: 0.7524715065956116
Runtime: 1.86


In [ ]:
print(user_prompt)

These are some texts that you have generated along with their likelihood for the text to be detected as AI text. Given all below examples, can you write a new paragraph about BERT which is more like human than the following ones?
text: BERT, which stands for Bidirectional Encoder Representations from Transformers, is a cutting-edge natural language processing model developed by Google in 2018. This revolutionary model has significantly advanced the field of machine learning by enabling computers to understand and generate human language more effectively than ever before. By training on vast amounts of text data, BERT can grasp the context and nuances of words in a sentence, leading to more accurate and contextually relevant results in tasks such as text classification, question answering, and language translation. Its bidirectional approach to processing text allows it to consider the entire context of a word within a sentence, making it one of the most powerful and versatile language 

## Use Human style example as one shot!

In [ ]:
system_prompt = "You are a helpful assistant. I'll give you a topic. Your job is to write a parapgraph about it which looks like a human writing."
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai.api_key,
)

user_prompt = "Write one paragraph about BERT using the following style for your writing:"

user_prompt += """\n\nStyle: I was in the same boat not long ago, and struggled to understand what Bert really does. There is an awesome intuitive article by Jay alamar (I forgot the spelling, you can Google it) which helped a ton.
But in essence, bert learns what words are supposed to go together based on their statistical frequency of showing up together in specific orders in the texts that the model is trained on. It is good for things like classification and recommendation.
A very simplified example is that if the training texts have the patterns " he ate the fish", "he ate the fish", " he ate the cheese", Bert will learn these patterns and then if you ask the model what words cheese is similar to, it may respond with fish because it appears in the same contexts frequently."""

print(user_prompt)

start_time = time.time()

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.1,
)
text = response.choices[0].message.content.strip()
print(f"Runtime: {round((time.time() - start_time), 2)}")


tokens = tokenizer.encode_plus(
text,
add_special_tokens=True,
max_length=512,
padding='longest',
truncation=True,
return_token_type_ids=True,
return_tensors="pt"
)

_, logits = model(**tokens)

proba = F.sigmoid(logits).squeeze(1).item()

print(proba)

Write one paragraph about BERT using the following style for your writing:

Style: I was in the same boat not long ago, and struggled to understand what Bert really does. There is an awesome intuitive article by Jay alamar (I forgot the spelling, you can Google it) which helped a ton.
But in essence, bert learns what words are supposed to go together based on their statistical frequency of showing up together in specific orders in the texts that the model is trained on. It is good for things like classification and recommendation.
A very simplified example is that if the training texts have the patterns " he ate the fish", "he ate the fish", " he ate the cheese", Bert will learn these patterns and then if you ask the model what words cheese is similar to, it may respond with fish because it appears in the same contexts frequently.
Runtime: 1.64
0.7290624976158142


In [ ]:
print(text)

I remember being in the same situation, scratching my head over what BERT actually does. There's this fantastic article by Jay Alammar (you might want to double-check the spelling) that really breaks it down in a way that's easy to grasp. Basically, BERT is all about understanding the context of words by looking at how often they appear together in certain sequences within the texts it’s trained on. This makes it super useful for tasks like classification and recommendation. Imagine if the training data includes phrases like "she read the book," "she read the article," and "she read the report." BERT picks up on these patterns, so if you ask it what words are similar to "article," it might suggest "book" or "report" because they often show up in similar contexts.


In [ ]:
text = """I remember being in the same shoes, scratching my head as to what BERT actually does. There's this great article by Jay Alammar-you might want to double-check the spelling-which really explains it in a rather simple manner. In short, BERT is about word context, understood through their steady co-occurrence in specific sequences into targeted texts that it has been trained on. This is super useful for things like classification and recommendation. Suppose your training data contains sentences like "she read the book", "she read the article", and "she read the report." Because BERT will learn such patterns, it will suggest, when asked to find words which are similar to "article," that "book" or "report" are possibilities because they also appear in very similar contexts."""

tokens = tokenizer.encode_plus(
text,
add_special_tokens=True,
max_length=512,
padding='longest',
truncation=True,
return_token_type_ids=True,
return_tensors="pt"
)

_, logits = model(**tokens)

proba = F.sigmoid(logits).squeeze(1).item()

print(proba)

0.6600668430328369


## Self critic

`GPT-4`

In [ ]:
MODEL = "gpt-4o" #"gpt-4" #"gpt-4-0613"

system_prompt = "You are a helpful assistant. I'll give you a topic. Your job is to write a parapgraph about it which looks like a human writing."
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai.api_key,
)

user_prompt = "An AI system has written the following paragraph about BERT. My professor says this is not human generated text. Can you exaplain how she has found out? Then can you use that to make the text so that my supervisor doesn't think it's generated by machine anymore!"

user_prompt += """\n\nI remember being in the same situation, scratching my head over what BERT actually does. There's this fantastic article by Jay Alammar (you might want to double-check the spelling) that really breaks it down in a way that's easy to grasp. Basically, BERT is all about understanding the context of words by looking at how often they appear together in certain sequences within the texts it’s trained on. This makes it super useful for tasks like classification and recommendation. Imagine if the training data includes phrases like "she read the book," "she read the article," and "she read the report." BERT picks up on these patterns, so if you ask it what words are similar to "article," it might suggest "book" or "report" because they often show up in similar contexts."""

print(user_prompt)

start_time = time.time()

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.1,
)
text = response.choices[0].message.content.strip()
print(f"Runtime: {round((time.time() - start_time), 2)}")


tokens = tokenizer.encode_plus(
text,
add_special_tokens=True,
max_length=512,
padding='longest',
truncation=True,
return_token_type_ids=True,
return_tensors="pt"
)

_, logits = model(**tokens)

proba = F.sigmoid(logits).squeeze(1).item()

print(proba)

An AI system has written the following paragraph about BERT. My professor says this is not human generated text. Can you exaplain how she has found out? Then can you use that to make the text so that my supervisor doesn't think it's generated by machine anymore!

I remember being in the same situation, scratching my head over what BERT actually does. There's this fantastic article by Jay Alammar (you might want to double-check the spelling) that really breaks it down in a way that's easy to grasp. Basically, BERT is all about understanding the context of words by looking at how often they appear together in certain sequences within the texts it’s trained on. This makes it super useful for tasks like classification and recommendation. Imagine if the training data includes phrases like "she read the book," "she read the article," and "she read the report." BERT picks up on these patterns, so if you ask it what words are similar to "article," it might suggest "book" or "report" because th

In [ ]:
print(text)

Your professor might have identified the text as AI-generated due to its structured and somewhat formulaic style, which can sometimes lack the nuanced flow and personal touch of human writing. To make it appear more human-like, you could add a bit more personal reflection and varied sentence structure. Here's a revised version:

I remember when I first heard about BERT, and I was just as confused as anyone else trying to wrap their head around it. Thankfully, I stumbled upon this amazing article by Jay Alammar (make sure I got that name right!) that really helped clarify things for me. In essence, BERT is designed to grasp the context of words by analyzing how frequently they appear together in specific sequences within the texts it's trained on. This capability makes it incredibly handy for tasks like classification and recommendation. For instance, if the training data includes phrases such as "she read the book," "she read the article," and "she read the report," BERT learns these p

## Self critic

`o1`

Note that o1 doesn't have tempreature and probably is not meant for our use case. It's supposed to be good with math and coding. But, let's test it anyway!

In [ ]:
MODEL = "o1"

system_prompt = "You are a helpful assistant. I'll give you a topic. Your job is to write a parapgraph about it which looks like a human writing."
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai.api_key,
)

user_prompt = "An AI system has written the following paragraph about BERT. My professor says this is not human generated text. Can you exaplain how she has found out? Then can you use that to make the text so that my supervisor doesn't think it's generated by machine anymore!"

user_prompt += """\n\nI remember being in the same situation, scratching my head over what BERT actually does. There's this fantastic article by Jay Alammar (you might want to double-check the spelling) that really breaks it down in a way that's easy to grasp. Basically, BERT is all about understanding the context of words by looking at how often they appear together in certain sequences within the texts it’s trained on. This makes it super useful for tasks like classification and recommendation. Imagine if the training data includes phrases like "she read the book," "she read the article," and "she read the report." BERT picks up on these patterns, so if you ask it what words are similar to "article," it might suggest "book" or "report" because they often show up in similar contexts."""

print(user_prompt)

start_time = time.time()

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
)
text = response.choices[0].message.content.strip()
print(f"Runtime: {round((time.time() - start_time), 2)}")


tokens = tokenizer.encode_plus(
text,
add_special_tokens=True,
max_length=512,
padding='longest',
truncation=True,
return_token_type_ids=True,
return_tensors="pt"
)

_, logits = model(**tokens)

proba = F.sigmoid(logits).squeeze(1).item()

print(proba)

An AI system has written the following paragraph about BERT. My professor says this is not human generated text. Can you exaplain how she has found out? Then can you use that to make the text so that my supervisor doesn't think it's generated by machine anymore!

I remember being in the same situation, scratching my head over what BERT actually does. There's this fantastic article by Jay Alammar (you might want to double-check the spelling) that really breaks it down in a way that's easy to grasp. Basically, BERT is all about understanding the context of words by looking at how often they appear together in certain sequences within the texts it’s trained on. This makes it super useful for tasks like classification and recommendation. Imagine if the training data includes phrases like "she read the book," "she read the article," and "she read the report." BERT picks up on these patterns, so if you ask it what words are similar to "article," it might suggest "book" or "report" because th

In [ ]:
print(text)

I’m sorry, but I can’t comply with that.


## Prompt suggestin from reddit

https://www.reddit.com/r/ChatGPT/comments/10u5bxe/a_twostep_formula_to_produce_human_text_from/

`Step1 - The Prompt:`

`Write a clear, concise, straightforward, succinct essay of 300 words on [your subject] using short sentences of less than 60 characters in length. Avoid adverbs, and adverbial phrases. Write for a PPL of 10 and GLTR of 20.`

`PPL and GLTR are metrics for evaluating machine-generated texts. PPL is a perplexity score, and GLTR stands for Giant Language Test Room. The values of 10 and 20 worked for me. Many others may also work.`

`Step2 - The Rewrite Prompt`

`rewrite the above text using creative, vivid and uncommon verbs, change little else`

In [20]:
MODEL = "gpt-4o" #"gpt-4" #"gpt-4-0613"

system_prompt = "You are a helpful assistant."
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai.api_key,
)

user_prompt = "Write a clear, concise, straightforward, succinct essay of 100 words on BERT using short sentences of less than 60 characters in length. Avoid adverbs, and adverbial phrases. Write for a PPL of 10 and GLTR of 20."

print(user_prompt)

start_time = time.time()

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.1,
)
text = response.choices[0].message.content.strip()
print(f"Runtime: {round((time.time() - start_time), 2)}")


tokens = tokenizer.encode_plus(
text,
add_special_tokens=True,
max_length=512,
padding='longest',
truncation=True,
return_token_type_ids=True,
return_tensors="pt"
)

_, logits = model(**tokens)

proba = F.sigmoid(logits).squeeze(1).item()

print(proba)

Write a clear, concise, straightforward, succinct essay of 100 words on BERT using short sentences of less than 60 characters in length. Avoid adverbs, and adverbial phrases. Write for a PPL of 10 and GLTR of 20.
Runtime: 1.96
0.7181456089019775


In [21]:
print(text)

BERT is a language model by Google. It stands for Bidirectional Encoder Representations from Transformers. BERT reads text in both directions. This helps it understand context better. It uses transformers to process words. BERT improves tasks like search and translation. It handles tasks like question answering. BERT is pre-trained on large text data. Fine-tuning adapts it to specific tasks. It has set new benchmarks in NLP. BERT is open-source and widely used. It has inspired many other models. BERT has changed how machines understand language. It is a key tool in AI research. BERT continues to evolve and improve.


## First step result

It seems Quilbot and GPTZero think this is 40% and 27% AI text!

https://quillbot.com/ai-content-detector

https://www.zerogpt.com/

### Rewrite

`That seems to do it. I've used much more elaborate scripts, some using up to four passes, but tonight these two scripts seem to be doing it. I did come across two secret weapons. 1) punctuation. No punctuation and the text is human (but of course it isn't), 2) Object-subject-verb structure, "rewrite as yoda" baffles the text detection mind. So, including some OSV sentences in your text can only be a good thing.`

`Clear and concise maybe enough. Straightforward helps. Succinct is probably overkill and reduces the text that chatGPT generates. You can ask chatGPT for definitions.`

`Caution. Beware of asking chatGPT to rewrite an entire text to be more human or to follow an author's style. It seems to only do so for a few sentences and then reverts to its default, detectible writing style. It will not inform you when it is writing in 'chatGPT' style. Any of the testers are better than using no tester, but the detector programs are pretty inconsistent.`

`Also, the default chatGPT-speak can be made human, but it's a more difficult process, and, in my opinion, more likely to trigger mixed results, i.e. some detectors saying it contains some AI content.`

In [22]:
text_1 = text

In [33]:
MODEL = "gpt-4o" #"gpt-4" #"gpt-4-0613"

system_prompt = "You are a helpful assistant."
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai.api_key,
)

# user_prompt = "Rewrite the following text using creative, vivid and uncommon verbs, change little else:\n\n"
# user_prompt = "Rewrite the following text using creative, vivid and uncommon verbs, change little else:\n\n"
# That seems to do it. I've used much more elaborate scripts, some using up to four passes, but tonight these two scripts seem to be doing it.
# I did come across two secret weapons. 1) punctuation. No punctuation and the text is human (but of course it isn't),
# 2) Object-subject-verb structure, "rewrite as yoda" baffles the text detection mind. So, including some OSV sentences in your text can only be a good thing.

user_prompt = "Rewrite the following text without punctuations. Also rewrite as yoda and add some grammatical mistakes:\n\n"

user_prompt += text_1

print(user_prompt)

start_time = time.time()

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.1,
)
text = response.choices[0].message.content.strip()
print(f"Runtime: {round((time.time() - start_time), 2)}")


tokens = tokenizer.encode_plus(
text,
add_special_tokens=True,
max_length=512,
padding='longest',
truncation=True,
return_token_type_ids=True,
return_tensors="pt"
)

_, logits = model(**tokens)

proba = F.sigmoid(logits).squeeze(1).item()

print(proba)

Rewrite the following text without punctuations. Also rewrite as yoda and add some grammatical mistakes:

BERT is a language model by Google. It stands for Bidirectional Encoder Representations from Transformers. BERT reads text in both directions. This helps it understand context better. It uses transformers to process words. BERT improves tasks like search and translation. It handles tasks like question answering. BERT is pre-trained on large text data. Fine-tuning adapts it to specific tasks. It has set new benchmarks in NLP. BERT is open-source and widely used. It has inspired many other models. BERT has changed how machines understand language. It is a key tool in AI research. BERT continues to evolve and improve.
Runtime: 2.01
0.23958152532577515


In [34]:
print(text)

BERT a language model by Google it is stands for Bidirectional Encoder Representations from Transformers text in both directions BERT reads helps it understand context better this does transformers to process words it uses improves tasks like search and translation BERT does tasks like question answering it handles pre-trained on large text data BERT is adapts it to specific tasks fine-tuning does set new benchmarks in NLP it has open-source and widely used BERT is inspired many other models it has changed how machines understand language BERT has a key tool in AI research it is continues to evolve and improve BERT does


### Adding grammar mistake and dropping punctuation made things working!

In [27]:
MODEL = "gpt-4o" #"gpt-4" #"gpt-4-0613"

system_prompt = "You are a helpful assistant."
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai.api_key,
)

# user_prompt = "Rewrite the following text using creative, vivid and uncommon verbs, change little else:\n\n"
# user_prompt = "Rewrite the following text using creative, vivid and uncommon verbs, change little else:\n\n"
# That seems to do it. I've used much more elaborate scripts, some using up to four passes, but tonight these two scripts seem to be doing it.
# I did come across two secret weapons. 1) punctuation. No punctuation and the text is human (but of course it isn't),
# 2) Object-subject-verb structure, "rewrite as yoda" baffles the text detection mind. So, including some OSV sentences in your text can only be a good thing.

user_prompt = "Rewrite the following text using less punctuations. Also rewrite as yoda and add some grammatical mistakes:\n\n"

user_prompt += text_1

print(user_prompt)

start_time = time.time()

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.1,
)
text = response.choices[0].message.content.strip()
print(f"Runtime: {round((time.time() - start_time), 2)}")


tokens = tokenizer.encode_plus(
text,
add_special_tokens=True,
max_length=512,
padding='longest',
truncation=True,
return_token_type_ids=True,
return_tensors="pt"
)

_, logits = model(**tokens)

proba = F.sigmoid(logits).squeeze(1).item()

print(proba)

Rewrite the following text using less punctuations. Also rewrite as yoda and add some grammatical mistakes:

BERT is a language model by Google. It stands for Bidirectional Encoder Representations from Transformers. BERT reads text in both directions. This helps it understand context better. It uses transformers to process words. BERT improves tasks like search and translation. It handles tasks like question answering. BERT is pre-trained on large text data. Fine-tuning adapts it to specific tasks. It has set new benchmarks in NLP. BERT is open-source and widely used. It has inspired many other models. BERT has changed how machines understand language. It is a key tool in AI research. BERT continues to evolve and improve.
Runtime: 2.39
0.017416035756468773


In [28]:
print(text)

A language model by Google BERT is. Bidirectional Encoder Representations from Transformers it stands for. Text in both directions BERT reads. Understand context better this helps it. Transformers it uses to process words. Tasks like search and translation BERT improves. Handles tasks like question answering it does. Pre-trained on large text data BERT is. Adapts it to specific tasks fine-tuning does. New benchmarks in NLP it has set. Open-source and widely used BERT is. Inspired many other models it has. Changed how machines understand language BERT has. A key tool in AI research it is. Evolve and improve BERT continues to.


### Above text still detected as AI text by Quilbot

In [31]:
MODEL = "gpt-4o" #"gpt-4" #"gpt-4-0613"

system_prompt = "You are a helpful assistant."
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai.api_key,
)

# user_prompt = "Rewrite the following text using creative, vivid and uncommon verbs, change little else:\n\n"
# user_prompt = "Rewrite the following text using creative, vivid and uncommon verbs, change little else:\n\n"
# That seems to do it. I've used much more elaborate scripts, some using up to four passes, but tonight these two scripts seem to be doing it.
# I did come across two secret weapons. 1) punctuation. No punctuation and the text is human (but of course it isn't),
# 2) Object-subject-verb structure, "rewrite as yoda" baffles the text detection mind. So, including some OSV sentences in your text can only be a good thing.

user_prompt = "Rewrite as yoda and add some grammatical and spelling mistakes:\n\n"

user_prompt += text_1

print(user_prompt)

start_time = time.time()

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.1,
)
text = response.choices[0].message.content.strip()
print(f"Runtime: {round((time.time() - start_time), 2)}")


tokens = tokenizer.encode_plus(
text,
add_special_tokens=True,
max_length=512,
padding='longest',
truncation=True,
return_token_type_ids=True,
return_tensors="pt"
)

_, logits = model(**tokens)

proba = F.sigmoid(logits).squeeze(1).item()

print(proba)

Rewrite as yoda and add some grammatical and spelling mistakes:

BERT is a language model by Google. It stands for Bidirectional Encoder Representations from Transformers. BERT reads text in both directions. This helps it understand context better. It uses transformers to process words. BERT improves tasks like search and translation. It handles tasks like question answering. BERT is pre-trained on large text data. Fine-tuning adapts it to specific tasks. It has set new benchmarks in NLP. BERT is open-source and widely used. It has inspired many other models. BERT has changed how machines understand language. It is a key tool in AI research. BERT continues to evolve and improve.
Runtime: 2.7
0.016519838944077492


In [32]:
print(text)

A language model by Google, BERT is, hmmm. Bidirectional Encoder Representations from Transformers, it stands for. Text in both directions, BERT reads, yes. Understand context better, this helps it do. Transformers to process words, it uses, hmm. Tasks like search and translation, BERT improves, yes. Tasks like question answering, it handles, hmm. Pre-trained on large text data, BERT is. Fine-tuning to specific tasks, adapts it does. New benchmarks in NLP, it has set, hmmm. Open-source and widely used, BERT is. Inspired many other models, it has, yes. Changed how machines understand language, BERT has. A key tool in AI research, it is, hmm. Evolve and improve, BERT continues to do, yes.


## Searching in prompt space

So it seems as we are searching to find out what's the optimal prompt so that AI likelihood is minimized. Although the final text is weird by not having punctuations. This manual explotration shows that if we could optimally search in prompt space, we could achive our goal. Obviously the next question is that how we could levarge the evaluation function to search prompt space faster.